In [ ]:
PROGRAM es_2
IMPLICIT NONE

!--------------------------------
!Code to numerically esteem erf(1) with two MC methods: Sample mean and importance sampling
!--------------------------------

LOGICAL, SAVE                      :: opt                 !Option to choose one method or another
                                                          !feature you can enable if needed
INTEGER, PARAMETER  :: nkind = 4                          !4 for single precision, 8 for double

REAL(kind=nkind), DIMENSION(:), ALLOCATABLE    :: rnd                            !Array random numbers
REAL(kind=nkind), PARAMETER                    :: pi = 3.14159265358979323846
REAL(kind=nkind), PARAMETER                    :: th = (sqrt(pi)/2)*erf(1.)      !Referement value                
INTEGER(kind=nkind)                            :: N, i, j, nrun                  !Number of points in sample and                                                                                        !number of runs and their counters
                                                                                                               
REAL(kind=nkind)                               :: F, sigma, x, G                 !Esteem variables


INTEGER                            :: sizer, values(1:8)                         !Seed variables
INTEGER, DIMENSION(:), ALLOCATABLE :: seed, box                                  !Seed variables



    



!--------------------------------------------
!Seed generation and saving
!--------------------------------------------

CALL random_seed(sizer)
ALLOCATE(seed(sizer))
ALLOCATE(box(sizer))       
CALL date_and_time(values=values)
seed(:) = values(8)

PRINT*,'Here the seed has ',sizer,'components'  
CALL random_seed(put=seed)
CALL random_seed(get=box)
OPEN (unit=1,file="seed.dat",status="replace",action="write")
DO i = 1 , sizer
   WRITE(unit=1,fmt=*)box(i)
END DO
CLOSE(1)
PRINT*,'seed data stored in "seed.dat"'




!--------------------------------------------
!SET INITIAL VALUE
!--------------------------------------------

PRINT*,'Numerical esteem of sqrt(pi)/2 *erf(1) by MC sample mean or importance sampling'
PRINT*,'Choose 0 for sample mean and 1 for importance sampling'


!CHOOSING METHOD FEATURE: ENABLE IF NEEDED
!READ*,i

!IF(i == 0)opt=.true.                   !set option for method
!IF(i == 1)opt=.false.



PRINT*, "Enter number of steps >"
READ*, N
PRINT*, "Enter number of runs >"
READ*, nrun
i = 0
j = 0

CALL plot(N, nrun, 10.)

OPEN (unit=7,file="es_2.dat",status="replace",action="write")




!-----------------------------------------
!SAMPLE MEAN METHOD
!-----------------------------------------

!IF(opt) THEN                                                                              !Choosing method feature
!    WRITE(7,*)'#SAMPLE MEAN INDEX'
!    WRITE(7,*)'# N, F_N, sigma_N, sigma_N/sqrt(N), abs error'
!    DO j = 1, nrun
!        ALLOCATE(rnd(j*N/nrun))
!        F = 0 !Set esteem
!        CALL random_number(rnd) 
!        F = SUM(exp(-rnd**2))/(j*N/nrun)
!        sigma = sqrt(SUM(exp(-rnd**2))/(j*N/nrun) - (SUM(exp(-rnd))/(j*N/nrun))**2)

!        WRITE(7,*)(j*N/nrun), F, sigma, sigma/sqrt(real((j*N/nrun))), abs(th - F)
!        DEALLOCATE(rnd)
!    END DO
!    WRITE(7,*)''
!    WRITE(7,*)''                                                                           !Space for GNUplot indices
!    PRINT*,'data saved in "es_2.dat"'
!END IF                                                                                    !Choosing method feature





!-----------------------------------------
!IMPORTANCE SAMPLING METHOD
!-----------------------------------------

!!IF(.not.opt) THEN                                                                       !Choosing method feature
!    WRITE(7,*)'#IMPORTANCE SAMPLING INDEX'
!    WRITE(7,*)'# N, F_N, sigma_N, sigma_N/sqrt(N), abs error'
!
!    DO j = 1, nrun
!        ALLOCATE(rnd(j*N/nrun))
!        i   = 1                                                     !Counter
!        F   = 0                                                     !Re-set the esteem
!
!        DO 
!            CALL expdev(x)                                          !points from p(x)=e^-x distribution
!            IF(x <= 1) THEN
!                rnd(i) = x
!                i = i + 1
!            END IF
!            IF(i == (j*N/nrun)+1) EXIT                              !Want to use N points exactly to compare with                                                                          !other method
!        END DO
!        F = SUM(EXP(-rnd**2)/EXP(-rnd))/(j*N/nrun) * (1 - EXP(-1.)) !Sum f(x)/p(x)*NORM
!        sigma = sqrt(SUM((EXP(-rnd**2)/EXP(-rnd))**2)/(j*N/nrun)&
!                - (SUM(EXP(-rnd**2)/EXP(-rnd))/(j*N/nrun))**2)
!
!        WRITE(7,*)(j*N/nrun), F, sigma, sigma/sqrt(real(j*N/nrun)), abs(th - F)
!        DEALLOCATE(rnd)
!    END DO
!    PRINT*,'data saved in "es_2.dat"'   
!!END IF                                                             !Choosing method feature

CLOSE(7)
STOP




CONTAINS

SUBROUTINE expdev(x)                                            !Subroutine to have point x from e^-x distribution
REAL(kind=nkind), INTENT(OUT) :: x
REAL(kind=nkind):: r
    DO
        CALL random_number(r)
        IF(r > 0) exit
    END DO
    x = -log(r)
END SUBROUTINE expdev

!---------------------------------------------------------
!Subroutine for log-log plot with equispaced data
!---------------------------------------------------------
SUBROUTINE plot(N, nrun, b)                                                    

REAL(kind=nkind)                             :: de                             !exponent interval for log-log plot
INTEGER(kind=nkind), INTENT(IN)              :: N, nrun                        !Total number of points and number
                                                                               !of points wanted in plot
REAL(kind=nkind), INTENT(IN)                 :: b                              !base for log-log plot

!Variables for sample mean
REAL(kind=nkind), DIMENSION(:), ALLOCATABLE  :: rnd                            !Array random numbers
REAL(kind=nkind), PARAMETER                  :: pi = 3.14159265358979323846    !Referement value
REAL(kind=nkind), PARAMETER                  :: th = (sqrt(pi)/2)*erf(1.)      !Referement value
INTEGER(kind=nkind)                          :: j
REAL(kind=nkind)                             :: F, sigma                       !Esteem variables


de = log10(real(N))/nrun

OPEN (unit=7,file="plot.dat",status="replace",action="write")
WRITE(7,*)'#PLOT SAMPLE MEAN'
WRITE(7,*)'# N, F_N, sigma_N, sigma_N/sqrt(N), abs error'
DO j = 1, nrun
    ALLOCATE(rnd(int(b**(j*de))))
    F = 0 !Set esteem
    CALL random_number(rnd) 
    F = SUM(exp(-rnd**2))/(int(b**(j*de)))
    sigma = sqrt(SUM(exp(-rnd**2))/(int(b**(j*de))) - SUM(exp(-rnd))/((int(b**(j*de)))**2))

    WRITE(7,*)(int(b**(j*de))), F, sigma, sigma/sqrt(real((int(b**(j*de))))), abs(th - F)
    DEALLOCATE(rnd)
END DO
WRITE(7,*)''
WRITE(7,*)''                                                                           !Space for GNUplot indices




WRITE(7,*)'#IMPORTANCE SAMPLING INDEX'
WRITE(7,*)'# N, F_N, sigma_N, sigma_N/sqrt(N), abs error'

DO j = 1, nrun
    ALLOCATE(rnd(int(b**(j*de))))
    i   = 1                                                     !Counter
    F   = 0                                                     !Re-set the esteem

    DO 
        CALL expdev(x)                                          !points from p(x)=e^-x distribution
        IF(x <= 1) THEN
            rnd(i) = x
            i = i + 1
        END IF
        IF(i == (int(b**(j*de)))+1) EXIT                              !Want to use N points exactly to compare with                                                                         !other method
    END DO
    F = SUM(EXP(-rnd**2)/EXP(-rnd))/(int(b**(j*de))) * (1 - EXP(-1.)) !Sum f(x)/p(x)*NORM
    sigma = sqrt(SUM((EXP(-rnd**2)/EXP(-rnd))**2)/(int(b**(j*de)))&
            - (SUM(EXP(-rnd**2)/EXP(-rnd))/(int(b**(j*de))))**2)

    WRITE(7,*)(int(b**(j*de))), F, sigma, sigma/sqrt(real(int(b**(j*de)))), abs(th - F)
    DEALLOCATE(rnd)
END DO
PRINT*,'Plot data saved in "plot.dat"'


    
END SUBROUTINE plot
END PROGRAM es_2










PROGRAM es_4
IMPLICIT NONE

!-------------------------------------------------------------------------------------------
!Code to numerically esteem pi with MC sample mean.
!2 possible variance reduction methods:
!option for average of averages method or option for subset division method. 
!-------------------------------------------------------------------------------------------


INTEGER, PARAMETER  :: nkind = 4                                               !4 for single precision, 8 for double
LOGICAL, SAVE       :: opt                                                     !Option to choose one method or another                                                                                !feature you can enable if needed

REAL(kind=nkind), DIMENSION(:), ALLOCATABLE   :: rnd                            !Array random numbers
REAL(kind=nkind), DIMENSION(:,:), ALLOCATABLE :: rnd_a                          !Array random numbers
REAL(kind=nkind), DIMENSION(:,:), ALLOCATABLE :: rnd_s                          !Array random numbers
REAL(kind=nkind), PARAMETER                   :: pi = 3.14159265358979323846    !Referement value           
INTEGER(kind=nkind)                           :: N, i, j, nrun                  !Number of points in sample and 
                                                                                !number of run and their counters  
INTEGER(kind=nkind)                           :: k                              !Number of points inside each subset
REAL(kind=nkind)                              :: F, sigma                       !Esteem variables
REAL(kind=nkind)                              :: sigmaM, M2, M                  !Esteem variables average of averages
REAL(kind=nkind)                              :: sigmaS, S2, S                  !Esteem variables subset
REAL(kind=nkind), PARAMETER                   :: b = 10                         !base for log-log plot


INTEGER                            :: sizer, values(1:8)                       !Seed variables
INTEGER, DIMENSION(:), ALLOCATABLE :: seed, box                                !Seed variables



    



!--------------------------------------------
!Seed generation and saving
!--------------------------------------------

CALL random_seed(sizer)
ALLOCATE(seed(sizer))
ALLOCATE(box(sizer))       
CALL date_and_time(values=values)
seed(:) = values(8)

PRINT*,'Here the seed has ',sizer,'components'  
CALL random_seed(put=seed)
CALL random_seed(get=box)
OPEN (unit=1,file="seed.dat",status="replace",action="write")
DO i = 1 , sizer
   WRITE(unit=1,fmt=*)box(i)
END DO
CLOSE(1)
PRINT*,'seed data stored in "seed.dat"'




!--------------------------------------------
!SET INITIAL VALUE
!--------------------------------------------

PRINT*,'Numerical esteem of pi by MC sample mean or importance sampling'
PRINT*,'Choose 0 for sample mean and 1 for importance sampling'


!CHOOSING METHOD FEATURE: ENABLE IF NEEDED
!READ*,i

!IF(i == 0)opt=.true.                   !set option for method
!IF(i == 1)opt=.false.



PRINT*, "Enter number of steps >"
READ*, N
PRINT*, "Enter number of runs >"
READ*, nrun
i  = 0
j  = 0
M  = 0
M2 = 0

!CALL plot(N, nrun, b)                        !Plot data before allocate rnd

ALLOCATE(rnd_a(nrun,N))
k = CEILING(REAL(N)/nrun)                    !CEILING returns the smallest integer geq what's inside  
ALLOCATE(rnd_s(nrun,k))   

OPEN (unit=7,file="es_4.dat",status="replace",action="write")


!-----------------------------------------
!AVERAGE OF AVERAGES METHOD
!-----------------------------------------

!IF(opt) THEN                                                   !Choosing method feature
    WRITE(7,*)'#AVERAGE OF AVERAGES INDEX'
    WRITE(7,*)'# nrun, N, F_N, sigma_N, sigma_N/sqrt(N), abs error'
    
    CALL random_number(rnd_a) 
    DO j = 1, nrun
        F = 0 !Set esteem
        F  = 4*SUM(sqrt(1-rnd_a(j,:)**2))/N                            !pi = 4 int_0^1 sqrt(1 - x**2) dx
        M  = M  + F                                             !Accumulate F and F**2
        M2 = M2 + F**2                                          
        sigma = sqrt((16*SUM(1-rnd_a(j,:)**2)/N) - (4*SUM(sqrt(1-rnd_a(j,:)**2))/N)**2)

        WRITE(7,*)j, N, F, sigma, sigma/sqrt(real(N)), abs(pi - F)
    END DO
    
    sigmaM = sqrt(M2/nrun - (M/nrun)**2)
    
    WRITE(7,*)''
    WRITE(7,*)''                                                !Space for GNUplot indices    
    WRITE(7,*)'#AVERAGE OF AVERAGES RESULTS'
    WRITE(7,*)'# N_tot, <M>, sigma_M, abs error'
    WRITE(7,*)nrun*N, M/nrun, sigmaM, abs(pi - M/nrun)
    WRITE(7,*)''
    WRITE(7,*)''                                                !Space for GNUplot indices
    PRINT*,'Average of averages data saved in "es_4.dat"'
!END IF                                                         !Choosing method feature




!-----------------------------------------
!SUBSET METHOD
!-----------------------------------------

!IF(opt) THEN                                                   !Choosing method feature
    WRITE(7,*)'#SUBSET METHOD INDEX'
    WRITE(7,*)'# nrun, N, F_N, sigma_N, sigma_N/sqrt(N), abs error'

    CALL random_number(rnd_s)
    IF(N/nrun == 0)PRINT*,"n subsets can't be greater than N"
    DO i = 1, nrun
        F = 0 !Set esteem
        F  = 4*SUM(sqrt(1-rnd_s(i,:)**2))/k                     !pi = 4 int_0^1 sqrt(1 - x**2) dx
        S  = S  + F                                             !Accumulate F and F**2
        S2 = S2 + F**2                                          
        sigma = sqrt((16*SUM(1-rnd_s(i,:)**2)/k) - (4*SUM(sqrt(1-rnd_s(i,:)**2))/k)**2)

        WRITE(7,*)i, k, F, sigma, sigma/sqrt(real(k)), abs(pi - F)
    END DO

    sigmaS = sqrt(S2/nrun - (S/nrun)**2) !Capire perché non viene giusto
    
    WRITE(7,*)''
    WRITE(7,*)''                                                !Space for GNUplot indices    
    WRITE(7,*)'#SUBSET METHOD RESULTS'
    WRITE(7,*)'# N_tot, <S>, sigma_S/sqrt(k), abs error'
    WRITE(7,*)nrun*k, S/nrun, sigmaS, abs(pi - S/nrun)
    WRITE(7,*)''
    WRITE(7,*)''                                                !Space for GNUplot indices
    PRINT*,'Subset method data saved in "es_4.dat"'
!END IF                                                         !Choosing method feature







CLOSE(7)
DEALLOCATE(rnd_a, rnd_s)
STOP




CONTAINS

!---------------------------------------------------------
!Subroutine for log-log plot with equispaced data
!---------------------------------------------------------
SUBROUTINE plot(N, nrun, b)                                                    

REAL(kind=nkind)                             :: de                             !exponent interval for log-log plot
INTEGER(kind=nkind), INTENT(IN)              :: N, nrun                        !Total number of points and number
                                                                               !of points wanted in plot
REAL(kind=nkind), INTENT(IN)                 :: b                              !base for log-log plot
!Variables for sample mean
REAL(kind=nkind), DIMENSION(:), ALLOCATABLE  :: rnd                            !Array random numbers
REAL(kind=nkind), PARAMETER                  :: pi = 3.14159265358979323846    !Referement value           
INTEGER(kind=nkind)                          :: j
REAL(kind=nkind)                             :: F, sigma                       !Esteem variables


de = log10(real(N))/nrun

OPEN (unit=7,file="plot.dat",status="replace",action="write")
WRITE(7,*)'#PLOT SAMPLE MEAN'
WRITE(7,*)'# N, F_N, sigma_N, sigma_N/sqrt(N), abs error'

DO j = 1, nrun
    ALLOCATE(rnd(int(b**(j*de))))
    F = 0                                                   !Re-Set esteem
    CALL random_number(rnd) 
    F = 4*SUM(sqrt(1 - rnd**2))/(b**(j*de))
    sigma = sqrt(SUM(1 - rnd**2)/(b**(j*de)) - (SUM(sqrt(1 - rnd**2))/(b**(j*de)))**2)

    WRITE(7,*)(int(b**(j*de))), F, sigma, sigma/sqrt(b**(j*de)), abs(pi - F)
    DEALLOCATE(rnd)
END DO
PRINT*,'Plot data saved in "plot.dat"'
    
END SUBROUTINE plot


END PROGRAM es_4




